# Le machine learning

- Machine Learning : Techniques de calcul permettant aux ordinateurs d’apprendre à exécuter des tâches en s’inspirant de patterns perçus sur des données, avec peu d’intervention humaine. Les tâches se résument souvent à de la prédiction.

![image](./images/ml1.png)
    

# Quelques exemples

- Prestation de service : identification des paramètres de prédisposition à la résiliation (churn)
- Marketing: identification de segments distincts de consommateurs 
- Production : identification de paramètres induisant des défaillances
- Pharmaceutique: classification de médicaments selon le profil chimique


# Le data mining vs le machine learning

- Data Mining : recherche d’information et de patterns cachés dans une base de données isolée ou en croisant plusieurs bases. C’est l’être humain qui explore et prend des décisions.
- Le data mining s’appuie sur des techniques et algorithmes issus de la statistique et de la gestion de bases de données.
- Les données exploitées ont souvent une taille considérable.


![image](./images/trend-ml.png)

# Les outils

![image](./images/poll-ml.png)

# Les types de machine learning

- Les algorithmes de machine learning peuvent être rassemblés dans 3 groupes principaux :
   - L'apprentissage supervisé
     - Cible (variable dépendante qualitative ou quantitative)
     - Prédicteurs (variables indépendantes)
     - Régression, arbre de decision, foret aléatoire, kNN, regression logistique, SVM…

   - L'apprentissage non supervisé
     - Pas de variable dépendante, pas de modélisation
     - Classer des individus dans différents groups ou trouver des associations
     - Apriori algorithm, K-means.

   - L'apprentissage par renforcement
     - Apprentissage pour prendre des décisions spécifique afin d’aboutir à un but précis
     - Markov Decision Process

# L’analyse de données et le machine learning avec Python

Le package la plus adaptée pour faire du data mining et du machine learning est la package **scikit-learn**


Ce package propose des fonctions prédéfinies pour un grand nombre de méthodes

- La classification : SVM, plus proches voisins, random forest…
- Les régressions : linéaire, ridge, Lasso…
- Le clustering : k-means…
- L’analyse de données :  ACP, DA…

Dans le cadre de cette formation, l'objectf n'est pas de décrire la théorie des méthodes mais plutôt de comprendre l'utilisation de Python pour les appliquer.

# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé telecom.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn=pd.read_csv("./data/telecom.csv")

In [3]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   object 
 5   VMail Plan      3333 non-null   object 
 6   VMail Message   3333 non-null   int64  
 7   Day Mins        3333 non-null   float64
 8   Day Calls       3333 non-null   int64  
 9   Day Charge      3333 non-null   float64
 10  Eve Mins        3333 non-null   float64
 11  Eve Calls       3333 non-null   int64  
 12  Eve Charge      3333 non-null   float64
 13  Night Mins      3333 non-null   float64
 14  Night Calls     3333 non-null   int64  
 15  Night Charge    3333 non-null   float64
 16  Intl Mins       3333 non-null   float64
 17  Intl Calls      3333 non-null   i

## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de trois colonnes object.

Nous pouvons afficher les statistiques descriptives pour les colonnes object :

In [4]:
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,351-9037,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les données sont toutes binaires. 

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables. Par
ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [5]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [6]:
# on définit les colonnes
# les colonnes quantitatives
col_quanti = ['Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls']

## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [7]:
x = churn[col_quanti]
y = churn["Churn?"]

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [8]:
# on importe la fonction
from sklearn.model_selection import train_test_split


Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.33, 
                                                    random_state=42)

In [10]:
assert x_train.shape[0] == y_train.shape[0]
print("Bien séparé !")

Bien séparé !


Ainsi les deux échantillons _train et _test ont la même distribution

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle GBM
```python
from sklearn.ensemble import GradientBoostingClassifier
```

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [12]:
modele_rf = RandomForestClassifier(n_estimators=100)
modele_knn = KNeighborsClassifier(n_neighbors=5)
modele_gbm = GradientBoostingClassifier()

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [13]:
modele_rf.fit(x_train,y_train)
modele_knn.fit(x_train,y_train)
modele_gbm.fit(x_train,y_train)
pass

Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [14]:
pd.DataFrame(modele_rf.feature_importances_,index=x.columns)

,0
Day Mins,0.147884
Day Calls,0.052154
Day Charge,0.160983
Eve Mins,0.084640
Eve Calls,0.049853
Eve Charge,0.081246
Night Mins,0.054861
Night Calls,0.052851
Night Charge,0.055032
Intl Mins,0.051735


Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [15]:
y_predict_rf = modele_rf.predict(x_test)
y_predict_knn = modele_knn.predict(x_test)
y_predict_gbm = modele_gbm.predict(x_test)

On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [16]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
accuracy_modele_gbm = accuracy_score(y_test,y_predict_gbm)
print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" %(accuracy_modele_knn))
print("Pourcentage de bien classés pour le modèle GBM :%.3f" %(accuracy_modele_gbm))

Pourcentage de bien classés pour le modèle RF : 0.905
Pourcentage de bien classés pour le modèle kNN :0.873
Pourcentage de bien classés pour le modèle GBM :0.904


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
confusion_matrix_gbm=confusion_matrix(y_test,y_predict_gbm)
print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
print("Matrice de confusion pour le modèle GBM :",
confusion_matrix_gbm, sep="\n")

Matrice de confusion pour le modèle RF :
[[923  17]
 [ 88  72]]
Matrice de confusion pour le modèle kNN :
[[923  17]
 [123  37]]
Matrice de confusion pour le modèle GBM :
[[921  19]
 [ 87  73]]


## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [18]:
from sklearn.metrics import roc_auc_score
auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])
auc_modele_gbm=roc_auc_score(y_test,modele_gbm.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
print("Aire sous la courbe ROC pour le modèle GBM :" ,auc_modele_gbm)


Aire sous la courbe ROC pour le modèle RF : 0.8797639627659575
Aire sous la courbe ROC pour le modèle kNN : 0.6991190159574467
Aire sous la courbe ROC pour le modèle GBM : 0.883530585106383


## La validation croisée
Jusqu’ici nous avons utilisé des indicateurs basés sur une seule occurrence de test. Ceci veut dire qu’on ne teste notre modèle que sur un seul échantillon.

Une approche alternative souvent utilisée est la validation croisée. Celle-ci est en fait basée sur la répétition de l’estimation et de la validation sur des données différentes.

Pour obtenir ce cv-score, on utilise :

In [19]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(modele_rf, x_train, y_train, cv=5, scoring='roc_auc')
scores_knn = cross_val_score(modele_knn, x_train, y_train, cv=5, scoring='roc_auc')
scores_gbm = cross_val_score(modele_gbm, x_train, y_train, cv=5, scoring='roc_auc')

print("AUC pour RF : %.3f (+/- %.3f)"% (scores_rf.mean(), scores_rf.std() * 2))
print("AUC pour kNN : %.3f (+/- %.3f)"% (scores_knn.mean(),scores_knn.std() * 2))
print("AUC pour GBM : %.3f (+/- %.3f)"% (scores_gbm.mean(),scores_gbm.std() * 2))


AUC pour RF : 0.850 (+/- 0.043)
AUC pour kNN : 0.622 (+/- 0.107)
AUC pour GBM : 0.844 (+/- 0.050)


## L’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [20]:
from sklearn.model_selection import GridSearchCV

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [21]:
dico_param= {"max_depth":[3,5,7,10], "n_estimators":[10,20,50,100]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [22]:
recherche_hyper = GridSearchCV(RandomForestClassifier(), 
                               dico_param, 
                               scoring="accuracy",cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [23]:
recherche_hyper.fit(x_train, y_train)
pass

In [24]:
print(recherche_hyper.best_params_)

{'max_depth': 10, 'n_estimators': 100}


In [25]:
print(recherche_hyper.best_score_)

0.9028169861859331


In [26]:
pd.DataFrame(recherche_hyper.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019546,0.000796,0.002205,0.000391,3,10,"{'max_depth': 3, 'n_estimators': 10}",0.865772,0.872483,0.863535,0.854260,0.867713,0.864753,0.006019,16
1,0.035908,0.001411,0.003193,0.000739,3,20,"{'max_depth': 3, 'n_estimators': 20}",0.872483,0.865772,0.865772,0.856502,0.869955,0.866097,0.005439,14
2,0.083763,0.002597,0.005397,0.000482,3,50,"{'max_depth': 3, 'n_estimators': 50}",0.865772,0.870246,0.868009,0.854260,0.872197,0.866097,0.006298,14
3,0.167953,0.005152,0.010363,0.001021,3,100,"{'max_depth': 3, 'n_estimators': 100}",0.868009,0.872483,0.868009,0.854260,0.874439,0.867440,0.007054,13
4,0.022142,0.000398,0.002393,0.000489,5,10,"{'max_depth': 5, 'n_estimators': 10}",0.874720,0.885906,0.899329,0.869955,0.899103,0.885803,0.012115,11
5,0.041681,0.000968,0.002792,0.000399,5,20,"{'max_depth': 5, 'n_estimators': 20}",0.883669,0.899329,0.894855,0.869955,0.887892,0.887140,0.010159,10
6,0.103937,0.003765,0.005771,0.000427,5,50,"{'max_depth': 5, 'n_estimators': 50}",0.888143,0.879195,0.892617,0.874439,0.890135,0.884906,0.006920,12
7,0.200669,0.003454,0.010174,0.000398,5,100,"{'max_depth': 5, 'n_estimators': 100}",0.888143,0.906040,0.901566,0.883408,0.903587,0.896549,0.009035,9
8,0.025133,0.000400,0.002393,0.000489,7,10,"{'max_depth': 7, 'n_estimators': 10}",0.897092,0.906040,0.894855,0.872197,0.917040,0.897445,0.014847,5
9,0.048462,0.001221,0.003192,0.000399,7,20,"{'max_depth': 7, 'n_estimators': 20}",0.897092,0.899329,0.897092,0.887892,0.901345,0.896550,0.004610,8


## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [27]:
from sklearn.externals import joblib
joblib.dump(recherche_hyper, './modele_grid.pkl')

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['./modele_grid.pkl']

Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [28]:
from sklearn.externals import joblib
#grid_search_mon_pipe = joblib.load('./data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
grid_search_mon_pipe.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.